In [1]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from interpreto.attributions.methods import (
    IntegratedGradients,
    KernelShap,
    Lime,
    OcclusionExplainer,
    Saliency,
    SmoothGrad,
    SobolAttribution,
)
from interpreto.attributions.methods.sobol_attribution import SobolIndicesOrders
from interpreto.commons.granularity import GranularityLevel
from interpreto.visualizations.attributions.classification_highlight import (
    MultiClassAttributionVisualization,
    SingleClassAttributionVisualization,
)

In [2]:
import pandas as pd

dataset_path = "/data/fanny.jourdan/interpreto_test/labeled_noise_text_dataset.txt"
model_name = "/data/fanny.jourdan/interpreto_test/distilbert_trivial_classifier"
tokenizer_name = "hf-internal-testing/tiny-random-distilbert"

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
df = pd.read_csv(dataset_path)

In [5]:
list_explainers = [
    OcclusionExplainer(model=model, batch_size=4, tokenizer=tokenizer, granularity_level=GranularityLevel.WORD),
    IntegratedGradients(model=model, batch_size=4, tokenizer=tokenizer, n_interpolations=10),
    SmoothGrad(model=model, batch_size=4, tokenizer=tokenizer, n_interpolations=50, noise_level=0.01),
    Saliency(model=model, batch_size=4, tokenizer=tokenizer),
    Lime(model=model, batch_size=4, tokenizer=tokenizer, n_perturbations=100, granularity_level=GranularityLevel.WORD),
    SobolAttribution(
        model=model,
        batch_size=4,
        tokenizer=tokenizer,
        n_token_perturbations=100,
        granularity_level=GranularityLevel.WORD,
        sobol_indices_order=SobolIndicesOrders.TOTAL_ORDER,
    ),
    KernelShap(
        model=model, batch_size=4, tokenizer=tokenizer, n_perturbations=100, granularity_level=GranularityLevel.WORD
    ),
]

k = 10

for explainer in list_explainers:
    print(f"Explaining with {explainer.__class__.__name__}")
    attribution_outputs = explainer.explain(model_inputs=[df["text"][k]])

    viz = SingleClassAttributionVisualization(
        attribution_output=attribution_outputs[0],
        css=".common-word-style { margin-right: 0.3em }",
    )
    viz.display()

Explaining with ClassificationOcclusionExplainer


/home/antonin.poche/interpreto/interpreto/commons/model_wrapping/inference_wrapper.py:262: UserWarning: Batch size of 14 is greater than the wrapper's batch size of 4. Consider adjust the batch size or the wrapper of split your data.
  warnings.warn(  # TODO: find the problem, it seems that there is no batching implemented


Explaining with ClassificationIntegratedGradients


Explaining with ClassificationSmoothGrad


Explaining with ClassificationSaliency


Explaining with ClassificationLime


/home/antonin.poche/interpreto/interpreto/commons/model_wrapping/inference_wrapper.py:262: UserWarning: Batch size of 100 is greater than the wrapper's batch size of 4. Consider adjust the batch size or the wrapper of split your data.
  warnings.warn(  # TODO: find the problem, it seems that there is no batching implemented


Explaining with ClassificationSobolAttribution


/home/antonin.poche/interpreto/.venv/lib/python3.12/site-packages/scipy/stats/_qmc.py:993: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)
/home/antonin.poche/interpreto/interpreto/commons/model_wrapping/inference_wrapper.py:262: UserWarning: Batch size of 1500 is greater than the wrapper's batch size of 4. Consider adjust the batch size or the wrapper of split your data.
  warnings.warn(  # TODO: find the problem, it seems that there is no batching implemented


Explaining with ClassificationKernelShap


In [6]:
k = 10

for explainer in list_explainers:
    print(f"Explaining with {explainer.__class__.__name__}")
    attribution_outputs = explainer.explain(model_inputs=[df["text"][k]], targets=torch.tensor([[0, 1]]))

    viz = MultiClassAttributionVisualization(
        attribution_output=attribution_outputs[0],
        class_names=["A", "B"],
        css=".common-word-style { margin-right: 0.3em }",
    )
    viz.display()

Explaining with ClassificationOcclusionExplainer


Explaining with ClassificationIntegratedGradients


Explaining with ClassificationSmoothGrad


Explaining with ClassificationSaliency


Explaining with ClassificationLime


Explaining with ClassificationSobolAttribution


Explaining with ClassificationKernelShap
